 Kenza Guermouch 932356421 
 Fatima Jebbari 932355441
 
 
 Importation des bibliothèques et initialisation

In [ ]:
import yfinance as yf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, DoubleType, LongType, StringType
from pyspark.sql.functions import col, count, when, lag, datediff, avg, stddev, min, max, corr
from pyspark.sql.functions import year, month, weekofyear
from pyspark.sql.window import Window

In [ ]:
# Initialiser Spark
spark = SparkSession.builder.appName("StockDataAnalysis").getOrCreate()

# Fonction pour lire les données

def download_stock_data(tickers, start_date, end_date):
    """Télécharge les données des actions via yfinance et les convertit en DataFrame Spark."""
    data = {}
    schema = StructType([
        StructField("Date", DateType(), True),
        StructField("Open", DoubleType(), True),
        StructField("High", DoubleType(), True),
        StructField("Low", DoubleType(), True),
        StructField("Close", DoubleType(), True),
        StructField("Adj Close", DoubleType(), True),
        StructField("Volume", LongType(), True),
        StructField("Ticker", StringType(), True)
    ])
    for ticker in tickers:
        df = yf.download(ticker, start=start_date, end=end_date)
        if df.empty:
            print(f"Aucune donnée pour {ticker}")
            continue
        df.reset_index(inplace=True)
        df['Ticker'] = ticker
        spark_df = spark.createDataFrame(df, schema=schema)
        data[ticker] = spark_df.dropna()
    return data

# Télécharger les données
start_date = "2020-01-01"
end_date = "2023-01-01"
tickers = ["AAPL", "MSFT", "GOOGL"]
stock_data = download_stock_data(tickers, start_date, end_date)

### Analyse des Données des Actions

Ce script permet d'analyser les données historiques des actions en utilisant Apache Spark et yfinance. Les fonctions implémentées permettent d'explorer, de comprendre et d'extraire des insights utiles des données. Voici un résumé des fonctionnalités principales :

1. **Exploration des Données :**
   - `show_first_last_rows`: Affiche les 40 premières et dernières lignes de chaque DataFrame pour avoir un aperçu des données.
   - `get_observation_count`: Compte le nombre total d'observations dans le DataFrame.
   - `deduce_periodicity`: Identifie la périodicité des points de données en calculant la différence entre les dates successives.
   - `descriptive_statistics`: Affiche les statistiques descriptives (moyenne, écart-type, min, max) pour toutes les colonnes.
   - `count_missing_values`: Comptabilise les valeurs manquantes pour chaque colonne.

2. **Corrélation :**
   - `calculate_correlation_all_pairs`: Calcule la corrélation entre toutes les paires d'actions pour une colonne donnée (par exemple, le prix de clôture).

3. **Calcul des Variations :**
   - `calculate_variations`: Ajoute des colonnes pour les variations quotidiennes et mensuelles des prix.

4. **Identification des Meilleures Actions :**
   - `best_stock_by_period`: Identifie l'action avec le meilleur rendement moyen pour une période donnée (mois ou année).

5. **Synthèse des Insights :**
   - `summarize_insights`: Compile tous les insights calculés (exploration, corrélations, variations) pour chaque action dans une structure lisible.

6. **Visualisation :**
   - `plot_variations`: Génère des graphiques pour visualiser les variations quotidiennes et mensuelles des prix des actions.

Le script est conçu pour fonctionner avec plusieurs tickers (par exemple, AAPL, MSFT, GOOGL) et fournit des visualisations interactives pour mieux comprendre les tendances et les performances des actions.

In [ ]:
# Fonction pour afficher les 40 premières et dernières lignes

def show_first_last_rows(spark_df):
    print("Premières 40 lignes :")
    spark_df.show(40)
    print("Dernières 40 lignes :")
    spark_df.orderBy(col("Date").desc()).show(40)

# Fonction pour obtenir le nombre d'observations

def get_observation_count(spark_df):
    count = spark_df.count()
    print(f"Nombre total d'observations : {count}")

# Fonction pour déduire la période entre les points de données

def deduce_periodicity(spark_df):
    window_spec = Window.orderBy("Date")
    spark_df = spark_df.withColumn("Difference", datediff(col("Date"), lag("Date", 1).over(window_spec)))
    periodicity = spark_df.select("Difference").distinct().collect()
    print("Périodes distinctes entre les points de données :", [row["Difference"] for row in periodicity])

# Fonction pour calculer les statistiques descriptives

def descriptive_statistics(spark_df):
    print("Statistiques descriptives :")
    spark_df.describe().show()

# Fonction pour compter les valeurs manquantes

def count_missing_values(spark_df):
    print("Valeurs manquantes par colonne :")
    missing = spark_df.select([count(when(col(c).isNull(), c)).alias(c) for c in spark_df.columns])
    missing.show()

# Fonction pour calculer la corrélation entre colonnes

def calculate_correlation(spark_df, col1, col2):
    correlation = spark_df.corr(col1, col2)
    print(f"Corrélation entre {col1} et {col2} : {correlation}")

# Appliquer les fonctions sur les données téléchargées
for ticker, df in stock_data.items():
    print(f"\nAnalyse pour {ticker} :")
    show_first_last_rows(df)
    get_observation_count(df)
    deduce_periodicity(df)
    descriptive_statistics(df)
    count_missing_values(df)
    calculate_correlation(df, "Open", "Close")

### Analyse des Prix et Rendements des Actions

Le code ci-dessus effectue une analyse approfondie des prix et des rendements des actions pour chaque ticker dans le jeu de données. Voici les principales étapes :

1. **Calcul des Moyennes des Prix :**
   - La fonction `calculate_average_prices` calcule les moyennes des prix d'ouverture et de clôture pour différentes périodes : annuelle, mensuelle et hebdomadaire. Les résultats sont affichés dans des tableaux pour chaque période.

2. **Calcul des Rendements Quotidiens :**
   - La fonction `calculate_daily_return` calcule le rendement quotidien pour chaque date en comparant le prix de clôture du jour courant avec celui du jour précédent. Ces rendements sont ajoutés sous forme d'une nouvelle colonne `Daily_Return`.

3. **Identification du Rendement Quotidien Maximum :**
   - La fonction `highest_daily_return` identifie la date, le ticker, et le rendement quotidien maximum pour l'ensemble des données.

4. **Calcul des Moyennes des Rendements :**
   - La fonction `calculate_average_returns` calcule les moyennes des rendements quotidiens pour différentes périodes : annuelle, mensuelle et hebdomadaire. Ces moyennes permettent d'identifier les tendances globales des rendements pour chaque action.

5. **Application des Fonctions :**
   - Pour chaque ticker, les fonctions ci-dessus sont appliquées en séquence afin de produire des insights détaillés sur les variations des prix et des rendements.

Ce code est utile pour comprendre les performances historiques des actions sur différentes périodes et identifier les moments de forte volatilité ou de rendements élevés.


In [ ]:
def calculate_average_prices(spark_df):
    print("Moyennes des prix d'ouverture et de clôture par période :")
    spark_df = spark_df.withColumn("Year", year(col("Date")))
    spark_df = spark_df.withColumn("Month", month(col("Date")))
    spark_df = spark_df.withColumn("Week", weekofyear(col("Date")))
    yearly_avg = spark_df.groupBy("Year").agg(avg("Open").alias("Avg_Open_Year"), avg("Close").alias("Avg_Close_Year"))
    monthly_avg = spark_df.groupBy("Year", "Month").agg(avg("Open").alias("Avg_Open_Month"), avg("Close").alias("Avg_Close_Month"))
    weekly_avg = spark_df.groupBy("Year", "Week").agg(avg("Open").alias("Avg_Open_Week"), avg("Close").alias("Avg_Close_Week"))
    yearly_avg.show()
    monthly_avg.show()
    weekly_avg.show()


#Calcul des rendements quotidiens
def calculate_daily_return(spark_df):
    window_spec = Window.orderBy("Date")
    spark_df = spark_df.withColumn(
        "Daily_Return",
        (col("Close") - lag("Close", 1).over(window_spec)) / lag("Close", 1).over(window_spec)
    )
    spark_df.select("Date", "Ticker", "Daily_Return").show(10)
    return spark_df

def highest_daily_return(spark_df):
    max_return = spark_df.orderBy(col("Daily_Return").desc()).select("Date", "Ticker", "Daily_Return").first()
    print(f"Action avec le rendement quotidien le plus élevé : {max_return}")

def calculate_average_returns(spark_df):
    spark_df = spark_df.withColumn("Year", year(col("Date")))
    spark_df = spark_df.withColumn("Month", month(col("Date")))
    spark_df = spark_df.withColumn("Week", weekofyear(col("Date")))
    yearly_avg_return = spark_df.groupBy("Year").agg(avg("Daily_Return").alias("Avg_Daily_Return_Year"))
    monthly_avg_return = spark_df.groupBy("Year", "Month").agg(avg("Daily_Return").alias("Avg_Daily_Return_Month"))
    weekly_avg_return = spark_df.groupBy("Year", "Week").agg(avg("Daily_Return").alias("Avg_Daily_Return_Week"))
    yearly_avg_return.show()
    monthly_avg_return.show()
    weekly_avg_return.show()


for ticker, df in stock_data.items():
    print(f"\nAnalyse pour {ticker} :")
    calculate_average_prices(df)
    df = calculate_daily_return(df)
    highest_daily_return(df)
    calculate_average_returns(df)

In [ ]:
# Fonction pour calculer la moyenne mobile

def calculate_moving_average(spark_df, column, period):
    """Ajoute une colonne de moyenne mobile au DataFrame Spark."""
    window_spec = Window.orderBy("Date").rowsBetween(-(period - 1), 0)
    spark_df = spark_df.withColumn(f"{column}_MA_{period}", avg(col(column)).over(window_spec))
    return spark_df

# Fonction pour calculer le rendement par période (semaine, mois, année)

def calculate_periodic_return(spark_df, period):
    """Calcule le rendement pour différentes périodes."""
    if period == "week":
        spark_df = spark_df.withColumn("Week", weekofyear(col("Date")))
        returns = spark_df.groupBy("Week").agg(avg("Daily_Return").alias("Avg_Weekly_Return"))
    elif period == "month":
        spark_df = spark_df.withColumn("Month", month(col("Date")))
        returns = spark_df.groupBy("Month").agg(avg("Daily_Return").alias("Avg_Monthly_Return"))
    elif period == "year":
        spark_df = spark_df.withColumn("Year", year(col("Date")))
        returns = spark_df.groupBy("Year").agg(avg("Daily_Return").alias("Avg_Yearly_Return"))
    else:
        raise ValueError("Invalid period. Choose from 'week', 'month', 'year'.")
    returns.show()

# Fonction pour identifier l'action avec le meilleur rendement pour une période donnée

def best_stock_by_period(stock_data, period):
    """Identifie l'action avec le meilleur rendement moyen dans une période donnée."""
    best_stocks = {}
    for ticker, spark_df in stock_data.items():
        spark_df = calculate_daily_return(spark_df)
        spark_df = spark_df.withColumn("Period", col("Date"))
        avg_return = spark_df.groupBy("Ticker").agg(avg("Daily_Return").alias("Avg_Return"))
        best_stocks[ticker] = avg_return.orderBy(col("Avg_Return").desc()).first()
    for ticker, result in best_stocks.items():
        print(f"Meilleure action pour {ticker} dans la période : {result}")

# Fonction pour visualiser les prix moyens d'ouverture et de clôture

def plot_average_prices(spark_df, ticker):
    pandas_df = spark_df.select("Year", "Month", "Week", "Open", "Close").toPandas()
    pandas_df = pandas_df.groupby(["Year", "Month"]).mean().reset_index()

    plt.figure(figsize=(12, 6))
    plt.plot(pandas_df.index, pandas_df["Open"], label="Prix Moyen Ouverture", marker="o")
    plt.plot(pandas_df.index, pandas_df["Close"], label="Prix Moyen Clôture", marker="o")
    plt.title(f"Prix Moyens d'Ouverture et de Clôture - {ticker}")
    plt.xlabel("Temps (Mois)")
    plt.ylabel("Prix")
    plt.legend()
    plt.show()

# Fonction pour visualiser les rendements quotidiens

def plot_daily_return(spark_df, ticker):
    pandas_df = spark_df.select("Date", "Daily_Return").toPandas()

    plt.figure(figsize=(12, 6))
    plt.plot(pandas_df["Date"], pandas_df["Daily_Return"], label="Rendement Quotidien", marker=".")
    plt.title(f"Rendement Quotidien - {ticker}")
    plt.xlabel("Date")
    plt.ylabel("Rendement Quotidien")
    plt.legend()
    plt.show()

In [ ]:
# Ajouter les colonnes nécessaires pour la visualisation
def add_time_columns(spark_df):
    """Ajoute les colonnes Year, Month et Week au DataFrame."""
    spark_df = spark_df.withColumn("Year", year(col("Date")))
    spark_df = spark_df.withColumn("Month", month(col("Date")))
    spark_df = spark_df.withColumn("Week", weekofyear(col("Date")))
    return spark_df

# Visualisation
for ticker, df in stock_data.items():
    print(f"\nVisualisation pour {ticker} :")
    df = add_time_columns(df)  # Ajouter les colonnes Year, Month, Week
    df = calculate_moving_average(df, "Close", 5)
    df = calculate_daily_return(df)
    plot_average_prices(df, ticker)
    plot_daily_return(df, ticker)

### Analyse et Interprétation des Graphiques

1. **Rendement Quotidien pour AAPL :**
   Ce graphique montre les variations quotidiennes du rendement de l'action AAPL entre 2020 et 2023. Nous pouvons observer des pics de rendement importants, ce qui reflète une volatilité élevée sur certaines périodes. Ces rendements quotidiens aident à comprendre la performance et le risque de cette action sur des horizons courts.

2. **Prix Moyens d'Ouverture et de Clôture pour MSFT :**
   Ce graphique illustre les tendances des prix moyens d'ouverture et de clôture de l'action MSFT sur une période mensuelle. Les deux courbes sont étroitement alignées, indiquant que les prix d'ouverture et de clôture évoluent de manière similaire. Une croissance constante est visible sur certaines périodes, suivie d'une stabilisation ou d'un déclin.

3. **Rendement Quotidien pour MSFT :**
   Le rendement quotidien de MSFT montre une volatilité relativement stable avec quelques fluctuations importantes. Ces informations sont cruciales pour identifier les jours de rendements exceptionnels et évaluer les risques associés à cette action.

4. **Prix Moyens d'Ouverture et de Clôture pour GOOGL :**
   Ce graphique représente les tendances des prix moyens d'ouverture et de clôture de GOOGL sur une base mensuelle. Une croissance significative est observée sur certaines périodes, suivie de corrections de marché. La cohérence entre les courbes d'ouverture et de clôture suggère une stabilité relative dans les variations journalières.

5. **Rendement Quotidien pour GOOGL :**
   Le graphique du rendement quotidien de GOOGL met en évidence des périodes de grande volatilité, souvent associées à des événements de marché ou des annonces spécifiques à l'entreprise. Ces informations sont utiles pour les investisseurs cherchant à comprendre les comportements journaliers de l'action.

Ces analyses visuelles complètent les calculs statistiques pour fournir une vue d'ensemble claire de la performance des actions et de leurs comportements respectifs sur des périodes données.


In [ ]:
def calculate_variations(spark_df):
    """Ajoute des colonnes pour les variations jour à jour et mois à mois."""
    window_spec = Window.orderBy("Date")
    spark_df = spark_df.withColumn("Daily_Change", col("Close") - lag("Close", 1).over(window_spec))
    spark_df = spark_df.withColumn("Monthly_Change", col("Close") - lag("Close", 30).over(window_spec))
    return spark_df

In [ ]:
def plot_variations(spark_df, ticker):
    pandas_df = spark_df.select("Date", "Daily_Change", "Monthly_Change").toPandas()

    plt.figure(figsize=(12, 6))
    plt.plot(pandas_df["Date"], pandas_df["Daily_Change"], label="Variation Quotidienne", marker=".")
    plt.plot(pandas_df["Date"], pandas_df["Monthly_Change"], label="Variation Mensuelle", marker=".")
    plt.title(f"Variations des Prix - {ticker}")
    plt.xlabel("Date")
    plt.ylabel("Variation")
    plt.legend()
    plt.show()

In [ ]:
# Visualisation et synthèse
for ticker, df in stock_data.items():
    print(f"\nVisualisation pour {ticker} :")
    df = calculate_variations(df)
    plot_variations(df, ticker)

### Variations des Prix pour AAPL, MSFT et GOOGL

Le graphique présente les variations quotidiennes et mensuelles des prix des actions AAPL, MSFT et GOOGL entre 2020 et 2023. Les variations quotidiennes (en bleu) restent relativement stables avec des fluctuations modérées, reflétant les mouvements réguliers des marchés financiers. En revanche, les variations mensuelles (en orange) montrent des pics significatifs, indiquant des tendances plus larges influencées par des événements majeurs, des annonces économiques ou des performances spécifiques à chaque entreprise. Ces graphiques permettent d'observer la volatilité à court terme et les changements plus marqués sur des périodes plus longues, offrant ainsi une vision complète des comportements des actions pour les investisseurs cherchant à comprendre leur performance et leur risque.


In [ ]:
# Fonction pour calculer la corrélation entre toutes les paires d'actions

def calculate_correlation_all_pairs(stock_data, column):
    """Calcule la corrélation entre toutes les paires d'actions pour une colonne donnée."""
    tickers = list(stock_data.keys())
    for i in range(len(tickers)):
        for j in range(i + 1, len(tickers)):
            stock1 = tickers[i]
            stock2 = tickers[j]
            df1 = stock_data[stock1].select("Date", col(column).alias(f"{column}_{stock1}"))
            df2 = stock_data[stock2].select("Date", col(column).alias(f"{column}_{stock2}"))
            joined_df = df1.join(df2, on="Date")
            correlation = joined_df.stat.corr(f"{column}_{stock1}", f"{column}_{stock2}")
            print(f"Corrélation entre {stock1} et {stock2} sur la colonne {column} : {correlation}")

# Exemple d'utilisation : Calcul de la corrélation entre toutes les paires d'actions pour la colonne 'Close'
calculate_correlation_all_pairs(stock_data, "Close")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_correlation_matrix(stock_data, column):
    """Trace une matrice de corrélation entre les colonnes des actions."""
    tickers = list(stock_data.keys())
    data_for_corr = []

    for ticker in tickers:
        df = stock_data[ticker].select("Date", col(column).alias(f"{ticker}_{column}")).toPandas()
        df.set_index("Date", inplace=True)
        data_for_corr.append(df)

    # Concaténer les colonnes des prix pour créer une matrice
    merged_data = pd.concat(data_for_corr, axis=1).dropna()

    # Calculer la matrice de corrélation
    correlation_matrix = merged_data.corr()

    # Visualiser la matrice de corrélation
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title(f"Matrice de corrélation pour {column}")
    plt.show()

# Exemple d'utilisation
plot_correlation_matrix(stock_data, "Close")

### Key Insights from Stock Data Analysis

1. **Highest Average Closing Price**: Among the analyzed stocks, the stock with the highest average closing price demonstrates strong and consistent performance, making it attractive for long-term investment.
2. **Highest Daily Volatility**: Identifying the most volatile stock provides insight into potential short-term trading opportunities and associated risks.
3. **Correlation Between Stocks**: The correlation matrix reveals how the selected stocks move together, helping in portfolio diversification and risk management.
4. **Best Performing Periods**: By analyzing returns over different periods, the most profitable times for investment in each stock are identified.
5. **Most Consistent Stock**: The stock with the least variability in returns showcases stability, suitable for risk-averse investors.
6. **Highest Traded Stock**: The stock with the highest average trading volume reflects strong market interest and liquidity.
7. **Highest Monthly Returns**: Pinpointing the months with the best performance assists in identifying seasonal trends and optimal entry points.
8. **Longest Growth Streak**: Observing the longest consecutive growth streak highlights consistent upward momentum in a stock's performance.

These insights collectively aid in understanding stock behavior, supporting better investment decisions, and offering a comprehensive view of the selected stock data.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure Daily_Return is calculated for all stocks before insights
def calculate_daily_return_all(stock_data):
    """Adds the Daily_Return column to all DataFrames in stock_data."""
    for ticker, df in stock_data.items():
        window_spec = Window.orderBy("Date")
        stock_data[ticker] = df.withColumn(
            "Daily_Return",
            (col("Close") - lag("Close", 1).over(window_spec)) / lag("Close", 1).over(window_spec)
        )
    print("Daily_Return column added to all stock DataFrames.")

# Call this function before performing insights
calculate_daily_return_all(stock_data)

# Insight 1: Stock with the highest average closing price
def highest_avg_close(stock_data):
    max_avg = None
    averages = []
    for ticker, df in stock_data.items():
        avg_close = df.select(avg("Close").alias("Avg_Close")).collect()[0]["Avg_Close"]
        averages.append((ticker, avg_close))
        if not max_avg or avg_close > max_avg[1]:
            max_avg = (ticker, avg_close)
    print(f"L'action avec le prix moyen de clôture le plus élevé est {max_avg[0]} avec {max_avg[1]:.2f}")

    # Visualization
    plt.figure(figsize=(10, 6))
    tickers, avg_closes = zip(*averages)
    plt.bar(tickers, avg_closes, color='blue', alpha=0.7)
    plt.title("Prix moyen de clôture pour chaque action")
    plt.ylabel("Prix moyen de clôture")
    plt.xlabel("Action")
    plt.show()
    return max_avg

# Insight 2: Stock with the highest daily return variability
def highest_volatility(stock_data):
    max_vol = None
    volatilities = []
    for ticker, df in stock_data.items():
        std_dev = df.select(stddev("Daily_Return").alias("Std_Dev")).collect()[0]["Std_Dev"]
        volatilities.append((ticker, std_dev))
        if not max_vol or std_dev > max_vol[1]:
            max_vol = (ticker, std_dev)
    print(f"L'action avec la plus grande volatilité quotidienne est {max_vol[0]} avec une std. dev. de {max_vol[1]:.4f}")

    # Visualization
    plt.figure(figsize=(10, 6))
    tickers, std_devs = zip(*volatilities)
    plt.bar(tickers, std_devs, color='orange', alpha=0.7)
    plt.title("Volatilité quotidienne (écart-type des rendements) pour chaque action")
    plt.ylabel("Volatilité quotidienne (std dev)")
    plt.xlabel("Action")
    plt.show()
    return max_vol

# Insight 3: Correlation matrix across all stocks
def calculate_correlation_matrix(stock_data, column):
    tickers = list(stock_data.keys())
    data_for_corr = []

    for ticker in tickers:
        df = stock_data[ticker].select("Date", col(column).alias(f"{ticker}_{column}")).toPandas()
        df.set_index("Date", inplace=True)
        data_for_corr.append(df)

    merged_data = pd.concat(data_for_corr, axis=1).dropna()
    correlation_matrix = merged_data.corr()
    print("Matrice de corrélation entre les stocks:")
    print(correlation_matrix)

    # Visualization
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title(f"Matrice de corrélation pour {column}")
    plt.show()

    return correlation_matrix

# Insight 4: Periods with highest overall returns
def best_period(stock_data):
    best_periods = {}
    for ticker, df in stock_data.items():
        df = df.withColumn("Year", year(col("Date")))
        yearly_return = df.groupBy("Year").agg(avg("Daily_Return").alias("Yearly_Return"))
        best_periods[ticker] = yearly_return.orderBy(col("Yearly_Return").desc()).first()
    for ticker, period in best_periods.items():
        print(f"Pour {ticker}, la meilleure année est {period['Year']} avec un rendement annuel moyen de {period['Yearly_Return']:.4f}")
    return best_periods

# Insight 5: Most consistent stock (lowest variability in returns)
def most_consistent_stock(stock_data):
    min_vol = None
    consistencies = []
    for ticker, df in stock_data.items():
        std_dev = df.select(stddev("Daily_Return").alias("Std_Dev")).collect()[0]["Std_Dev"]
        consistencies.append((ticker, std_dev))
        if not min_vol or std_dev < min_vol[1]:
            min_vol = (ticker, std_dev)
    print(f"L'action la plus constante est {min_vol[0]} avec une std. dev. de {min_vol[1]:.4f}")

    # Visualization
    plt.figure(figsize=(10, 6))
    tickers, std_devs = zip(*consistencies)
    plt.bar(tickers, std_devs, color='green', alpha=0.7)
    plt.title("Constance des actions (écart-type des rendements)")
    plt.ylabel("Volatilité quotidienne (std dev)")
    plt.xlabel("Action")
    plt.show()
    return min_vol

# Insight 6: Stock with highest total volume traded
def highest_traded_stock(stock_data):
    max_volume = None
    volumes = []
    for ticker, df in stock_data.items():
        total_volume = df.select(avg("Volume").alias("Total_Volume")).collect()[0]["Total_Volume"]
        volumes.append((ticker, total_volume))
        if not max_volume or total_volume > max_volume[1]:
            max_volume = (ticker, total_volume)
    print(f"L'action la plus échangée est {max_volume[0]} avec un volume moyen de {max_volume[1]:.2f}")

    # Visualization
    plt.figure(figsize=(10, 6))
    tickers, avg_volumes = zip(*volumes)
    plt.bar(tickers, avg_volumes, color='purple', alpha=0.7)
    plt.title("Volume moyen échangé pour chaque action")
    plt.ylabel("Volume moyen")
    plt.xlabel("Action")
    plt.show()
    return max_volume

# Apply and summarize insights
print("\n--- Calculating Insights with Visualizations ---")
highest_avg_close(stock_data)
highest_volatility(stock_data)
correlation_matrix = calculate_correlation_matrix(stock_data, "Close")
best_period(stock_data)
most_consistent_stock(stock_data)
highest_traded_stock(stock_data)


### Analyse des graphiques

1. **Matrice de corrélation pour les prix de clôture des actions :**  
   Ce graphique présente une matrice de corrélation entre les prix de clôture des actions AAPL, MSFT et GOOGL. Les corrélations sont toutes très élevées, montrant des valeurs allant de **0.86 à 0.97**, ce qui indique que les actions évoluent de manière similaire, probablement influencées par des tendances communes du marché.

2. **Volatilité quotidienne (écart-type des rendements) :**  
   Ce graphique en barres montre l'écart-type des rendements quotidiens pour chaque action. AAPL présente la volatilité la plus élevée (**0.020**), suivie de GOOGL et MSFT qui sont légèrement inférieurs. Cela indique qu'AAPL est la plus volatile des trois actions, impliquant des fluctuations de prix plus importantes au quotidien.

3. **Volume moyen échangé pour chaque action :**  
   Ce graphique met en évidence le volume moyen des transactions pour AAPL, MSFT et GOOGL. AAPL domine largement avec un volume moyen supérieur à **100 millions**, tandis que MSFT et GOOGL sont beaucoup plus bas. Cela suggère qu'AAPL est l'action la plus activement échangée, ce qui pourrait être un indicateur de sa popularité parmi les investisseurs.

4. **Prix moyen de clôture pour chaque action :**  
   Ce graphique en barres illustre le prix moyen de clôture pour chaque action sur la période étudiée. MSFT a le prix moyen de clôture le plus élevé (environ **250 USD**), suivi par AAPL et GOOGL. Cela reflète la perception de la valeur marchande de ces actions sur la période donnée.

5. **Volatilité quotidienne (récapitulation avec une échelle différente) :**  
   Une autre visualisation des écarts-types des rendements pour réitérer que toutes les actions affichent des niveaux de volatilité relativement proches, bien qu'AAPL reste légèrement plus volatile.

Ces analyses permettent de mieux comprendre les dynamiques des actions AAPL, MSFT et GOOGL, notamment en termes de corrélations, de volumes échangés, de prix moyens et de volatilité, fournissant ainsi une base solide pour des décisions d'investissement.
